<a href="https://colab.research.google.com/github/SangamSilwal/DeepLearning_DSeries/blob/main/T8_CNN_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/content/fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [4]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train = X_train/255.0
X_test = X_test/255.0

In [14]:
class CustomDataset(Dataset):

  def __init__(self,features,labels):
    self.features = torch.tensor(features,dtype=torch.float32).reshape(-1,1,28,28)  #reshape(-1,1,28,28) -> -1 is for batch size, 1 -> because it is a gray scale image
    self.labels = torch.tensor(labels,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self,index):
    return self.features[index], self.labels[index]

In [15]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [19]:

class CNN(nn.Module):
  def __init__(self,input_features):
    super().__init__()

    self.features = nn.Sequential(
        nn.Conv2d(input_features,32,kernel_size=(3,3),padding='same'),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2),stride=2),

        nn.Conv2d(32,64,kernel_size=(3,3),padding='same'),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2),stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7,128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        nn.Linear(64,10)

    )

  def forward(self,x):
    x = self.features(x)
    x = self.classifier(x)
    return x

In [21]:
learning_rate = 0.01
epochs = 20

In [22]:
model = CNN(1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [24]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


Epoch: 1 , Loss: 1.4218302146593729
Epoch: 2 , Loss: 0.9588696948687235
Epoch: 3 , Loss: 0.7617603435118994
Epoch: 4 , Loss: 0.6596534313758214
Epoch: 5 , Loss: 0.593578049937884
Epoch: 6 , Loss: 0.5263444776336352
Epoch: 7 , Loss: 0.48378332873185476
Epoch: 8 , Loss: 0.4407579788565636
Epoch: 9 , Loss: 0.4063006140788396
Epoch: 10 , Loss: 0.37831186592578886
Epoch: 11 , Loss: 0.34967163592576983
Epoch: 12 , Loss: 0.32166188925504685
Epoch: 13 , Loss: 0.29880090574423473
Epoch: 14 , Loss: 0.2729723677535852
Epoch: 15 , Loss: 0.2650318702061971
Epoch: 16 , Loss: 0.2539433194200198
Epoch: 17 , Loss: 0.2314780028661092
Epoch: 18 , Loss: 0.2261434552570184
Epoch: 19 , Loss: 0.21496886442104976
Epoch: 20 , Loss: 0.19028132242461046
